In [23]:
import base64
import subprocess
import pandas as pd
import csv
import os
import re
import sys
import subprocess

from datetime import datetime
from paramiko.client import SSHClient
from paramiko.sftp_client import SFTPClient
from paramiko import AutoAddPolicy
from dateutil.parser import parse
from tqdm import tqdm

df = pd.DataFrame()

In [24]:
remote_user = 'django'
host = 'ambition-test.bhp.org.bw'
path = os.path.expanduser('/home/django/source/ambition/logs/access.log')
timestamp = datetime.today().strftime('%Y%m%d%H%M%S')

In [25]:
pat = (r''
       '(\d+.\d+.\d+.\d+)\s-\s-\s' #IP address
       '\[(.+)\]\s' #datetime
       '"GET\s(.+)\s\w+/.+"\s' #requested file
       '(\d+)\s' #status
       '(\d+)\s' #bandwidth
       '"(.+)"\s' #referrer
       '"(.+)"' #user agent
    )

def blocks(files, size=65536):
    while True:
        b = files.read(size)
        if not b: break
        yield b

def connect(ssh=None):
    ssh.load_system_host_keys()
    ssh.set_missing_host_key_policy(AutoAddPolicy())
    ssh.connect(
        host,
        username=remote_user,
        timeout=5,
        # banner_timeout=self.banner_timeout,
        compress=True,
    )
    return ssh

In [26]:
lines = []
from pprint import pprint
df = pd.DataFrame([], columns=['ip', 'datestring', 'url', 'status', 't', 'host', 'client'])
index = 0
with SSHClient() as ssh:
    ssh = connect(ssh=ssh)
    with SFTPClient.from_transport(ssh.get_transport()) as sftp_client:
        with sftp_client.open(path, mode='r') as f:
            total = sum(bl.decode('utf-8').count('\n') for bl in blocks(f))
            f.seek(0, 0)
            for line in tqdm(f, total=total):
                match = re.findall(pat, line)
                df_ln = pd.DataFrame(
                    list(match), columns=['ip', 'datestring', 'url', 'status', 't', 'host', 'client'])
                df = pd.concat([df, df_ln], ignore_index=True)

49369it [08:30, 96.70it/s]                            


In [27]:
# load from csv
# filename = '~/ambition-test-bhp-org-bw-access-log-raw-20171018091600.csv'
# df = pd.read_csv(os.path.expanduser(filename), low_memory=False)

In [28]:
df.to_csv(os.path.expanduser(f'~/{host.replace(".", "-")}-access-log-raw-{timestamp}.csv'))
print(f'~/{host.replace(".", "-")}-access-log-raw-{timestamp}.csv')

~/ambition-test-bhp-org-bw-access-log-raw-20171018092210.csv


In [100]:
df.info()

In [101]:
df.head()

In [ ]:
# convert datestring to date
def to_datetime(d):
    d = d.split('/')
    d = ' '.join([d[1], d[0], d[2]]).split(':')
    d = d[0] + ' ' + ':'.join([d[1], d[2], d[3].replace('+0200', '')])
    return parse(timestr=d)

df['date'] = df.apply(lambda row: to_datetime(row['datestring']), axis=1)
df['date'] = df['date'].astype('datetime64[ns]')

# chop off querystring
df['simple_url'] = df.apply(lambda row: row['url'].split('?')[0], axis=1)

# remove junk urls
df = df[
   (-df['simple_url'].isin(['/', '/admin/jsi18n/']))
   & (-df['simple_url'].str.contains('static'))
   & (-df['simple_url'].str.contains('login'))]

# show df info
df.info()

In [ ]:
# review min max date
df['date'].describe()

In [89]:
# rank users effort
from datetime import datetime, date
now = date.today()
today = datetime(now.year, now.month, now.day, 0, 0, 0)
df[df['date'] >= today].groupby('ip').size()

# remove logins, static, etc
df1 = df[(-df['simple_url'].isin(['/', '/admin/jsi18n/']))
   & (-df['simple_url'].str.contains('static'))
   & (-df['simple_url'].str.contains('login'))]

In [90]:
startdate = datetime(2017, 10, 10)
df1['date'] = df1['date'].dt.normalize()
df1 = df1[df1['date'] >= startdate]

In [91]:
# timestamp = datetime.today().strftime('%Y%m%d%H%M%S')
# df1.to_csv(os.path.expanduser(f'~/{host.replace(".", "-")}-access-log-{timestamp}.csv'))

### Activity in hits per IP

In [92]:
from dateutil.relativedelta import relativedelta

grouped = df1.groupby(['date'])
for date, _ in grouped.groups.items():
    print(date)
    print(df1[
        (df1['date'] >= date)
        & (df1['date'] < (date + relativedelta(days=1)))].groupby('ip').size())
# print(today - relativedelta(days=2))
# print(df1[
#     (df1['date'] >= today - relativedelta(days=2))
#     & (df1['date'] < today - relativedelta(days=1))].groupby('ip').size())
# print(today - relativedelta(days=1))
# print(df1[
#     (df1['date'] >= today - relativedelta(days=1))
#     & (df1['date'] < today)].groupby('ip').size())
# print(today)
# print(df1[df1['date'] >= today].groupby('ip').size())

2017-10-10 00:00:00
ip
10.113.201.72    3
172.31.6.51      1
dtype: int64
2017-10-11 00:00:00
ip
10.113.201.114    11
10.113.201.204    10
10.136.211.206    97
172.31.6.51       69
dtype: int64
2017-10-12 00:00:00
ip
10.113.201.159     3
10.113.201.186     5
10.113.201.2      17
10.113.201.233     4
10.136.211.206     1
10.136.211.70      8
dtype: int64
2017-10-13 00:00:00
ip
10.113.201.136      2
10.113.201.189     23
10.113.201.196      9
10.136.211.206    123
dtype: int64
2017-10-14 00:00:00
ip
10.113.200.115    1
dtype: int64
2017-10-16 00:00:00
ip
10.113.201.102     2
10.136.211.74      6
10.136.211.78     71
dtype: int64
2017-10-17 00:00:00
ip
10.113.201.118    4
10.113.201.124    5
dtype: int64
2017-10-18 00:00:00
ip
10.113.201.107    13
10.113.201.180    15
dtype: int64


In [93]:
# inspect top user
# selected_ips = ['10.136.211.78']
# df1 = df1[df1['ip'].isin(selected_ips)]

### Type of activity

#### Add

In [94]:
# add models
add_group = df1[(df1['simple_url'].str.contains('add'))
                & (-df1['simple_url'].str.contains('appointment'))
               ].groupby(['simple_url'])
add_group.size()

simple_url
/admin/ambition_subject/adverseevent/add/                                   3
/admin/ambition_subject/adverseeventfollowup/add/                           5
/admin/ambition_subject/bloodresult/add/                                    7
/admin/ambition_subject/clinicnote/add/                                    16
/admin/ambition_subject/lumbarpuncturecsf/add/                              4
/admin/ambition_subject/microbiology/add/                                   3
/admin/ambition_subject/patienthistory/add/                                13
/admin/ambition_subject/prnmodel/add/                                      15
/admin/ambition_subject/radiology/add/                                      1
/admin/ambition_subject/subjectconsent/add/                                 9
/admin/ambition_subject/subjectrequisition/add/                             2
/admin/ambition_subject/subjectscreening/add/                              11
/admin/ambition_subject/subjectvisit/add/            

#### Edit / Change

In [95]:
# change models
df1[(df1['simple_url'].str.contains('change'))
    & (-df1['simple_url'].str.contains('appointment'))
   ].groupby(['simple_url']).size()

simple_url
/admin/ambition_subject/adverseevent/aa54fddb-3716-4927-934f-23af141650d7/change/         3
/admin/ambition_subject/bloodresult/ef82059f-7d34-4fb7-a69e-f200a2a518a1/change/          1
/admin/ambition_subject/lumbarpuncturecsf/203a8c25-ec00-4f4b-bc00-2d7374aa2397/change/    2
/admin/ambition_subject/patienthistory/92f8cca8-0d37-4056-8d95-8ea5010b45fa/change/       1
/admin/ambition_subject/prnmodel/368c3016-c0cf-4914-bad8-5d80c4732262/change/             4
/admin/ambition_subject/prnmodel/39c3c590-cd94-4c2a-8d2b-79649fbc4b73/change/             2
/admin/ambition_subject/prnmodel/c2802920-221b-4568-b3b5-903ccd0fd416/change/             1
/admin/ambition_subject/radiology/a4de55b4-18ee-4f0e-8a3e-7d7f130aca9c/change/            1
/admin/ambition_subject/subjectconsent/6b261388-071c-4f74-bccc-27e1e6f57130/change/       2
/admin/ambition_subject/subjectconsent/c2b19369-c892-49d3-8ff0-e5d7aa66a68b/change/       2
/admin/ambition_subject/subjectscreening/33e1ea45-a554-4c95-ade3-3dc9

#### Delete

In [96]:
# delete models
df1[(df1['simple_url'].str.contains('delete'))
    & (-df1['simple_url'].str.contains('appointment'))
    ].groupby(['simple_url']).size()

Series([], dtype: int64)

####  Appointments

In [59]:
# accessing appointments
df1[(df1['simple_url'].str.contains('appointment'))].groupby(['simple_url']).size()

simple_url
/admin/ambition_subject/appointment/                                                  8
/admin/ambition_subject/appointment/00542ee2-b9e9-4624-a700-0711af34e7ba/change/      2
/admin/ambition_subject/appointment/019fe083-4ec8-403b-98c2-fcd586400b2f/change/      2
/admin/ambition_subject/appointment/04160671-d695-4934-8a06-feb46625b9bf/change/      2
/admin/ambition_subject/appointment/049c72f7-b106-4714-ae57-484cd53b5dad/change/      8
/admin/ambition_subject/appointment/04b390ee-61c3-4f83-a8d4-28e6bd267f74/change/      3
/admin/ambition_subject/appointment/062f42c2-ac4f-47ba-b354-747c7600d768/change/      2
/admin/ambition_subject/appointment/065ecaa3-6d45-4fb8-8d0b-4af875187505/change/      2
/admin/ambition_subject/appointment/077badb1-584f-4e9b-8c2a-2458ba41947a/change/      7
/admin/ambition_subject/appointment/084d6d16-6db1-446b-bec5-1b22e8c5098d/change/      3
/admin/ambition_subject/appointment/08fdd5aa-df41-494e-a8e7-54f189d21b74/change/      8
/admin/ambition_subje

#### Dashboard

In [102]:
# accessing dashboards
df1[(df1['simple_url'].str.contains('dashboard'))
   & (-df1['simple_url'].str.contains('_dashboard'))].groupby(['simple_url']).size()

simple_url
/subject/dashboard/092-40990004-7/                                          2
/subject/dashboard/092-40990004-7/86962bc1-7d04-4290-a4ff-9e9929877c3e/     2
/subject/dashboard/092-40990008-8/                                          4
/subject/dashboard/092-40990008-8/5d612563-ffad-47b7-a378-557e566e9550/     3
/subject/dashboard/092-40990009-6/                                          1
/subject/dashboard/092-40990009-6/049c72f7-b106-4714-ae57-484cd53b5dad/     1
/subject/dashboard/092-40990010-4/                                          4
/subject/dashboard/092-40990010-4/17bc94bb-a2d1-4ed0-96e9-a18d26c25803/     3
/subject/dashboard/092-40990011-2/                                         39
/subject/dashboard/092-40990011-2/464de491-5812-44be-b92b-a069d47af4c3/     8
/subject/dashboard/092-40990011-2/bbab7e54-1702-494d-a120-88c93378520b/     4
/subject/dashboard/092-40990012-0/                                         22
/subject/dashboard/092-40990012-0/21f42dca-4e15-4509-

#### Screening Listboards

In [103]:
# accessing listboards
df1[(df1['simple_url'].str.contains('screening_listboard'))].groupby(['simple_url']).size()

simple_url
/subject/screening_listboard/             29
/subject/screening_listboard/1/            1
/subject/screening_listboard/2/            1
/subject/screening_listboard/3/            1
/subject/screening_listboard/S9943ER9/     1
/subject/screening_listboard/S99A9AKP/     1
/subject/screening_listboard/S99WMDW9/     3
dtype: int64

#### Pharmacy

In [108]:
df1[(df1['simple_url'].str.contains('edc_pharma'))].groupby(['simple_url']).size()

simple_url
/edc_pharma_dashboard/                                               4
/edc_pharma_dashboard/listboard/dispense/                            7
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990007-0/    2
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990010-4/    2
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990011-2/    2
dtype: int64

#### Lab

In [109]:
df1[(df1['simple_url'].str.contains('edc_lab'))].groupby(['simple_url']).size()

simple_url
/edc_lab_dashboard//    1
dtype: int64

#### Edc

In [110]:
# accessing edc modules
df1[(df1['simple_url'].str.contains('edc'))].groupby(['simple_url']).size()

simple_url
/edc/settings/                                                       1
/edc_device/                                                         2
/edc_identifier/admin/                                               1
/edc_lab_dashboard//                                                 1
/edc_pharma_dashboard/                                               4
/edc_pharma_dashboard/listboard/dispense/                            7
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990007-0/    2
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990010-4/    2
/edc_pharma_dashboard/listboard/dispensetimepoint/092-40990011-2/    2
dtype: int64